In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv("ml-1m//movies.dat", sep ="::", engine="python", header = None, names=["MovieID","Title","Genres"] )
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings =  pd.read_csv("ml-1m//ratings.dat", sep ="::", engine="python", header = None, names=["UserID","MovieID","Rating","Timestamp"])
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
users = pd.read_csv("ml-1m/users.dat", #tab do podpowiadania sciezek do plikow
            sep="::",
            engine="python",
            header=None,
            names=["UserID", "Gender", "Age", "Occupation", "Zip-code"]) 
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


## Rok produkcji filmu

In [5]:
# slice
# regex

movies["Year"] = movies.Title.str[-5:-1]
movies.Title = movies.Title.str[:-7]
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 4 columns):
MovieID    3883 non-null int64
Title      3883 non-null object
Genres     3883 non-null object
Year       3883 non-null object
dtypes: int64(1), object(3)
memory usage: 121.4+ KB


## Ilość filmów

In [6]:
len(movies)

3883

## Ile filmów powstało w poszczegołnych latach

In [7]:
movies.Year.value_counts().reset_index().rename({"index" : "Year", "Year" : "Count"},axis =1).sort_values(by="Year").head()

,Year,Count
77,1919,3
79,1920,2
80,1921,1
78,1922,2
75,1923,3


## jak wygląda rozkład płci oraz grup wiekowych wśród użytkowników

In [8]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [9]:
pd.crosstab(users.Age,users.Gender)

Gender,F,M
Age,,
1,78,144
18,298,805
25,558,1538
35,338,855
45,189,361
50,146,350
56,102,278


## jaki gatunek filmowy jest najczęstszy

In [10]:
movies.head()

,MovieID,Title,Genres,Year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama,1995
4,5,Father of the Bride Part II,Comedy,1995


In [11]:
movies.Genres.str.split("|", expand = True).stack().reset_index(drop=True).value_counts()

Drama          1603
Comedy         1200
Action          503
Thriller        492
Romance         471
Horror          343
Adventure       283
Sci-Fi          276
Children's      251
Crime           211
War             143
Documentary     127
Musical         114
Mystery         106
Animation       105
Fantasy          68
Western          68
Film-Noir        44
dtype: int64

## jaki jest najlepszy film wszechczasów, (najlepszy, czyli ma najwyższą średnią ocenę) - to zadanie możesz rozwiązać wykonując złączenie (join) zbioru movies i rating

In [12]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [13]:
wynik = pd.merge(movies,ratings,on="MovieID")
pd.pivot_table(wynik[["Title","Rating"]], index="Title",aggfunc="mean").sort_values(by="Rating",ascending=False).head(10)

,Rating
Title,
Smashing Time,5.0
Song of Freedom,5.0
One Little Indian,5.0
Ulysses (Ulisse),5.0
"Baby, The",5.0
Follow the Bitch,5.0
Schlafes Bruder (Brother of Sleep),5.0
"Gate of Heavenly Peace, The",5.0
Bittersweet Motel,5.0


## wykonaj poprzedni punkt, odrzucając wcześniej filmy które nie uzyskały wystarczająco dużo głosów (np 100)

In [14]:
srednieOceny = ratings.groupby("MovieID").agg({"Rating":["mean","size"]})
srednieOceny100 = srednieOceny[srednieOceny[("Rating", "size")] > 100]
srednieOceny100.columns = ["mean", "size"]
pd.merge(srednieOceny100, movies, left_index=True, right_on="MovieID").sort_values(by="mean", ascending = False)[0:1]

,mean,size,MovieID,Title,Genres,Year
1950,4.56051,628,2019,Seven Samurai (The Magnificent Seven) (Shichin...,Action|Drama,1954


## jaki jest najlepszy film według kobiet i według mężczyzn

In [15]:
ratings_users = pd.merge(ratings, users)
srednieOcenyPlcie = ratings_users.groupby(["MovieID", "Gender"]).agg({"Rating" : ["mean", "size"]})
srednieOcenyPlcie.columns = ["mean", "size"]
srednieOcenyPlcie = srednieOcenyPlcie.reset_index()
srednieOcenyPlcie100 = srednieOcenyPlcie[srednieOcenyPlcie["size"] > 100]
pd.merge(srednieOcenyPlcie100, movies).sort_values(by="mean", ascending=False).drop_duplicates(subset="Gender")

,MovieID,Gender,mean,size,Title,Genres,Year
474,745,F,4.644444,180,"Close Shave, A",Animation|Comedy|Thriller,1995
521,858,M,4.583333,1740,"Godfather, The",Action|Crime|Drama,1972


## jaki jest średni rok oglądanego filmu w poszczególnych grupach wiekowych

In [24]:
everything = pd.merge(pd.merge(movies,ratings),users)

#to powinno działać, ale nie działa

0          1995
1          1995
2          1995
3          1977
4          1993
5          1993
6          1992
7          1937
8          1991
9          1996
10         1996
11         1996
12         1995
13         1996
14         1964
15         1939
16         1958
17         1950
18         1964
19         1941
20         1965
21         1982
22         1975
23         1987
24         1962
25         1989
26         1985
27         1959
28         1996
29         1997
           ... 
1000179    1999
1000180    1999
1000181    1999
1000182    1999
1000183    1999
1000184    1999
1000185    1999
1000186    1999
1000187    1999
1000188    1999
1000189    1999
1000190    1999
1000191    1999
1000192    1999
1000193    1999
1000194    1999
1000195    2000
1000196    2000
1000197    2000
1000198    2000
1000199    2000
1000200    2000
1000201    2000
1000202    2000
1000203    2000
1000204    2000
1000205    2000
1000206    2000
1000207    2000
1000208    2000
Name: Year, Length: 1000

,MovieID,Title,Genres,Year,UserID,Rating,Timestamp,Gender,Age,Occupation,Zip-code
0,1,Toy Story,Animation|Children's|Comedy,1995,1,5,978824268,F,1,10,48067
1,48,Pocahontas,Animation|Children's|Musical|Romance,1995,1,5,978824351,F,1,10,48067
2,150,Apollo 13,Drama,1995,1,5,978301777,F,1,10,48067
3,260,Star Wars: Episode IV - A New Hope,Action|Adventure|Fantasy|Sci-Fi,1977,1,4,978300760,F,1,10,48067
4,527,Schindler's List,Drama|War,1993,1,5,978824195,F,1,10,48067
5,531,"Secret Garden, The",Children's|Drama,1993,1,4,978302149,F,1,10,48067
6,588,Aladdin,Animation|Children's|Comedy|Musical,1992,1,4,978824268,F,1,10,48067
7,594,Snow White and the Seven Dwarfs,Animation|Children's|Musical,1937,1,4,978302268,F,1,10,48067
8,595,Beauty and the Beast,Animation|Children's|Musical,1991,1,5,978824268,F,1,10,48067
9,608,Fargo,Crime|Drama|Thriller,1996,1,4,978301398,F,1,10,48067
